In [2]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-08-23 16:40:31'

- **@author**: run_walker
- **@references**: 
    1. [Python Solution beats 99.80%](https://leetcode.com/problems/count-primes/discuss/134754/Python-Solution-beats-99.80)

# 问题
统计所有小于非负整数$n$的质数的数量。

**示例：**输入10，输出4，因为小于 10 的质数一共有 4 个，分别为2、3、5、7。

# python实现
假设小于$n$的质数一共有$i$个，依次记为$a_1,~...,~a_i$。

对比思路一和思路二，思路一要从2遍历到$n-1$，遇到合数找到第一个质因子会跳到下一个数进行判断，遇到质数$k$时需要试遍2到$\lfloor\sqrt k\rfloor$中的所有质数；而思路二只需要从2遍历到$\lfloor\sqrt n\rfloor$，遇到合数会直接跳过，遇到质数$k$时只需要标记从$k^2$到$n$中$k$的倍数为合数。思路二的效率要高于效率一。

## 法一 
依次从2到$n-1$进行遍历，分别判断每个数是否为质数。单独判断一个数$x$是否是质数的方式为：从2到$\lfloor\sqrt x\rfloor$进行遍历，判断其中是否有$x$的因子，事实上只要判断其中的质数是否能成为$x$的因子即可，因为合数一定能分解为更小的质因子相乘，因为要统计小于$n$的质数的数量，所以在判断$x$是否为质数时可以知道之前的质数有哪些。

In [3]:
def countPrimes_method1(n):
    if n <= 2:
        return 0, []
    zhishu = []
    for i in range(2, n):
        flag = 0
        ub = int(i**0.5)
        for j in zhishu:
            if j > ub:
                break
            if not i % j:
                flag = 1
                break
        if flag == 0:
            zhishu.append(i)
    return len(zhishu), zhishu

In [4]:
for i in range(12):
    l, zhishu = countPrimes_method1(i)
    print(f'{i}: {l}, {zhishu}')

0: 0, []
1: 0, []
2: 0, []
3: 1, [2]
4: 2, [2, 3]
5: 2, [2, 3]
6: 3, [2, 3, 5]
7: 3, [2, 3, 5]
8: 4, [2, 3, 5, 7]
9: 4, [2, 3, 5, 7]
10: 4, [2, 3, 5, 7]
11: 4, [2, 3, 5, 7]


## 法二：埃拉托斯特尼筛法
[维基百科 > Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) 

<img src="imgs/Sieve_of_Eratosthenes_animation.gif" style="float:left"></img>

质数的倍数（超过一倍）一定为合数，合数$x$的质因子只可能为从2到$\lfloor\sqrt x\rfloor$中的质数。因此我们可以从2出发向右遍历质数，将它们的倍数标记为合数，最后剩下来未被标记为合数的均为质数。

**注意：**
1. 合数的倍数没有必要标记为合数，因为遍历到它的质因子时已经标记过了。
2. 对于质数$k$，以及$c < k$，没有必要标记$k*c$为合数，因为遍历到$c$的质因子时已经标记过了。

In [5]:
def countPrimes_method2(n):
    if n <= 2: return 0, []
    flags = [0, 0] + [1] * (n - 2)
    for i in range(2, int(n**0.5) + 1):
        if flags[i]:
            flags[i * i: n: i] = [0] * ((n - 1) // i - i + 1)
    return sum(flags), [i for i, flag in enumerate(flags) if flag]

In [6]:
for i in range(12):
    l, zhishu = countPrimes_method2(i)
    print(f'{i}: {l}, {zhishu}')

0: 0, []
1: 0, []
2: 0, []
3: 1, [2]
4: 2, [2, 3]
5: 2, [2, 3]
6: 3, [2, 3, 5]
7: 3, [2, 3, 5]
8: 4, [2, 3, 5, 7]
9: 4, [2, 3, 5, 7]
10: 4, [2, 3, 5, 7]
11: 4, [2, 3, 5, 7]


### 改进
在内存中存储一个全局变量，记录已经判断过的质数，当输入的n在此范围内时直接返回，否则在原来的基础上继续向后判断。当多次调用该函数进行判断时，可以略提升一点效率。

In [21]:
flags = [0, 0]
def countPrimes_method3(n):
    global flags
    if n <= 2: return 0, []
    l = len(flags)
    if n > l:
        flags += [1] * (n - l)
        for i in range(2, int(n**0.5) + 1):
            if flags[i]:
                temp = max(i*i, l // i * i)
                flags[temp: n: i] = [0] * ((n - temp - 1) // i + 1)
    return sum(flags), [i for i, flag in enumerate(flags[:n]) if flag]

并没有写成递归的方式，所以没法通过`functools.lru_cache(None)`设置缓存进行加速。

In [24]:
for i in range(20):
    l, zhishu = countPrimes_method3(i)
    print(f'{i}: {l}, {zhishu}')

0: 0, []
1: 0, []
2: 0, []
3: 4, [2]
4: 4, [2, 3]
5: 4, [2, 3]
6: 4, [2, 3, 5]
7: 4, [2, 3, 5]
8: 4, [2, 3, 5, 7]
9: 4, [2, 3, 5, 7]
10: 4, [2, 3, 5, 7]
11: 4, [2, 3, 5, 7]
12: 5, [2, 3, 5, 7, 11]
13: 5, [2, 3, 5, 7, 11]
14: 6, [2, 3, 5, 7, 11, 13]
15: 6, [2, 3, 5, 7, 11, 13]
16: 6, [2, 3, 5, 7, 11, 13]
17: 6, [2, 3, 5, 7, 11, 13]
18: 7, [2, 3, 5, 7, 11, 13, 17]
19: 7, [2, 3, 5, 7, 11, 13, 17]


# 相关题目
- [leetcode 204. Count Primes](https://leetcode-cn.com/problems/count-primes/)
- [leetcode 1175. 质数排列]()